In [1]:
dir_data = '../data'
dir_ckpt = '../ckpt'

config = {
    'seed': 0,
    'k': 0.5,
    'epochs': 500,
    'hidden_dim': 10,
    'last_data': '2022-11-15',
    
    'ids': range(0, 21),
    
    'features': ['temperature', 'humidity', 'dew_point', 'wind_dir', 'wind_spd',
                 'uv_idx', 'visibility', 'ceiling', 'cloudiness', 'precip_prob',
                 'precip_1h', 'forecast', 'capacity', 'hour', 'year', 'month', 'day'],
    
    'optimizer': {
        'lr': 0.0,
        'weight_decay': 0.01
    },

    'scheduler': {
        'T_0': 500,
        'T_mult': 1,
        'eta_max': 0.001,
        'T_up': 10,
        'gamma': 0.5,
    },

    'loss': {
        'loss': 'MeanStdLoss',
        'k': 1.0,
        'weights': (1.0, 0.0, 0.0),
    },

    'wandb': {
        'project': 'solar_power_prediction',
        'name': 'WaveNet'
    }
}


In [2]:
import sys
sys.path.append('../')

from suncapturer.utils import seed_all
from suncapturer.solver import Trainer
from suncapturer.datasets import SolarDataset
from suncapturer.networks import WaveNet, CausalWaveNet

import os
import numpy as np
import pandas as pd
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from sklearn.preprocessing import MinMaxScaler, minmax_scale


In [3]:
seed_all(config['seed'])

run_id = int(datetime.timestamp(datetime.now()))
config['run_id'] = run_id
config['dir_ckpt'] = os.path.join(dir_ckpt, str(run_id))

In [4]:
df = pd.read_csv(os.path.join(dir_data, 'forecasts1_16.csv'))
df = df.fillna(0)

start = datetime.strptime('2020-09-30', '%Y-%m-%d')
df['fcst_time'] = df['fcst_time'].str.split(' ').str[0]
df['fcst_time'] = pd.to_datetime(df['fcst_time'])
df['fcst_time'] = (df['fcst_time'] - start).dt.days

valid_start = df['fcst_time'].max() - 14
train_df = df[(df['id'].isin(config['ids'])) & (df['fcst_time'] < valid_start)].copy()
valid_df = df[(df['id'].isin(config['ids'])) & (df['fcst_time'] >= valid_start)].copy()
test_df = df[(df['fcst_time'] >= valid_start) & (df['id'] == 0)].copy()
config['valid_start'] = valid_start

scaling_factor = {}
target_columns = df.columns.difference(['id', 'fcst_time', 'time', 'amount', 'relative_amount'])
for col in target_columns:
    scaler = MinMaxScaler()
    scaler.fit(train_df[col].values.reshape(-1, 1))
    scaling_factor[col] = (scaler.data_min_.item(), scaler.data_max_.item())
    
    train_df[col] = scaler.transform(train_df[col].values.reshape(-1, 1))
    valid_df[col] = scaler.transform(valid_df[col].values.reshape(-1, 1))
    test_df[col] = scaler.transform(test_df[col].values.reshape(-1, 1))

os.makedirs(config['dir_ckpt'])

with open(os.path.join(config['dir_ckpt'], 'scaling_factor.pkl'), 'wb') as f:
    import pickle
    pickle.dump(scaling_factor, f)

In [5]:
train_dataset = SolarDataset(train_df, config['features'])
valid_dataset = SolarDataset(valid_df, config['features'])
test_dataset = SolarDataset(test_df, config['features'])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [6]:
model = WaveNet(len(config['features']), config['hidden_dim'])
trainer = Trainer(model, config)
trainer.fit(train_loader, valid_loader, test_loader)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dongjinlee. Use `wandb login --relogin` to force relogin


/home/dongjin/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/modules/conv.py:303: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:883.)
  return F.conv1d(input, weight, bias, self.stride,


Epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
LR,▃███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
test_Loss,▆▆▅▇▁▆▇▃▂▆▄▃▃▆▅▄▃▄▃█▅▃▄▅▄▄▅▆▄▆▅▄▄▅▄▅▅▄▄▅
test_Metric1,▆▆▅▇▁▆▇▃▂▆▄▃▃▆▅▄▃▄▃█▅▃▄▅▄▄▅▆▄▆▅▄▄▅▄▅▅▄▄▅
test_Metric2,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_Metric3,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_TotalMetric,▆▆▅▇▁▆▇▃▂▆▄▃▃▆▅▄▃▄▃█▅▃▄▅▄▄▅▆▄▆▅▄▄▅▄▅▅▄▄▅
train_Loss,█▃▂▂▃▂▂▂▂▂▂▁▂▁▂▁▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_Metric1,█▃▂▂▃▂▂▂▂▂▂▁▂▁▂▁▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_Metric2,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_Metric3,▅▇█▇██▇▇▇▇▇█▇███▇██▇▇█▇▇▇███▇█▇▇██████▁█
